A code-first introduction to Stable Diffusion using the Diffusers library.

# Importing Libraries


In [ ]:
!pip install -Uq diffusers transformers fastcore

In [ ]:
import logging
from pathlib import Path

import matplotlib.pyplot as plt
import torch
from diffusers import StableDiffusionPipeline
from fastcore.all import concat
from huggingface_hub import notebook_login
from PIL import Image

logging.disable(logging.WARNING)

torch.manual_seed(1)
if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

In [ ]:
def image_grid(imgs, rows, cols):
    w,h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    for i, img in enumerate(imgs): grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

# Stable Diffusion

Before diving into the code, let's first take a bird's eye view of how Stable Diffusion works. Stable Diffusion is a latent diffusion algorithm, meaning it works with the images in latent space, that is compresses the images into a smaller dimesnion using an autoencoder. This makes the process both faster and memory efficient. In the forward process of diffusion, Gaussian noise is added to the images using an encoder over a number of steps till the images is indistinguishable from white noise. In the backward process the images are iteratively denoised to generate new samples. The denoising is done using a unet model that predicts and subtracts noise which is then subtracted from the image.

![](Diffusion.png)

# The Diffusion Pipeline

Diffusers library by Hugging Face is arguably the best way to use Stable Diffusion. The Diffusers library uses `StableDiffusionPipeline`, which is an end-to-end diffusion inference pipeline to combine all the steps in the diffusion process to easily do inference with the model. However, in this tutorial we will not use the pipeline directly ourselves but rather build it ourselves from its components to get a better understanding of stable diffusion works under the hood. This also gives us more control over the output generated by the model.


In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", variant="fp16", torch_dtype=torch.float16).to("cuda")

# Looking inside the pipeline

The diffusion pipeline combines all the components of stable diffusion for convenient inference. Let's build the pipeline back by putting together its components.

The main componets of a latent diffusion algorithm are:

1. An autoencoder (VAE)- for compressing the image in the latent space
2. A u-net- for predicting noise in the latent image
3. A text encoder: for creating embeddings of text and images.  

The output of a u-net, the noise residual, is used for gradual denoising of the latent image representation via a scheduler algorithm.


In [ ]:
del pipe

First, we need a text tokenizer and text encoder. Stable Diffusion uses the CLIP encoder by Open AI, so we'll be using its weights:


In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer

In [ ]:
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16).to("cuda")

Next, we'll use the `vae` and `unet` models.


In [ ]:
from diffusers import AutoencoderKL, UNet2DConditionModel

In [ ]:
# Here we use a different VAE to the original release, which has been fine-tuned for more steps
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-ema", torch_dtype=torch.float16).to("cuda")
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", torch_dtype=torch.float16).to("cuda")

The standard pipeline used the PNDM scheduler, but we'll use the K-LMS scheduler.
We need to use the same noising schedule as was used during training. The schedule is defined by the time step at which the noise is added and the amount of noise that is added, which is given by the beta parameters.
For the K-LMS scheduler, this is how the betas evolve:


In [ ]:
beta_start, beta_end = 0.00085, 0.012
plt.plot(torch.linspace(beta_start ** 0.5, beta_end ** 0.5, 1000) ** 2)
plt.xlabel('Timestep')
plt.ylabel('β')

In [ ]:
from diffusers import LMSDiscreteScheduler

In [ ]:
scheduler = LMSDiscreteScheduler(beta_start=beta_start, beta_end=beta_end, beta_schedule='scaled_linear', num_train_timesteps=1000)

We'll now define the parameters we'll use for generation


In [ ]:
prompt = "Batman drinking coffee while sitting in a cafe"

height = 512
width = 512
num_inference_steps = 70
guidance_scale = 7.5
batch_size = 1

Next, we tokenize the prompt. To ensure that we have same number of tokens for each prompt, we'll use max length padding and truncation.


In [ ]:
text_input = tokenizer(prompt, padding='max_length', max_length=tokenizer.model_max_length, truncation=True, return_tensors='pt')
text_input['input_ids']

In [ ]:
tokenizer.decode(49407), tokenizer.decode(49406)

In [ ]:
text_input['attention_mask']

The text encoder gives us the embeddings for the text prompt we have used.


In [ ]:
text_embeddings = text_encoder(text_input.input_ids.to('cuda'))[0].half()
text_embeddings.shape

We'll also get the embeddings required for unconditional generation, which is achieved with an empty string; the model is free to go anywhere as long as the results as reasonable. These embeddings will be used for classifier-free guidance.


In [ ]:
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(['']*batch_size, padding='max_length',max_length=max_length, return_tensors='pt')
uncond_embeddings = text_encoder(uncond_input.input_ids.to('cuda'))[0].half()
uncond_embeddings.shape

For classifier-free guidance, we need to do two forward passes. One with the conditioned input (`text_embeddings`), and another with the unconditional embeddings (`uncond_embeddings`). In practice, we can concatenate both into a single batch to avoid doing two forward passes.


In [ ]:
text_embeddings = torch.cat([text_embeddings, uncond_embeddings])
text_embeddings.shape

To start the denoising process, we'll start from pure Gaussian noise. These are our initial latents.


In [ ]:
torch.manual_seed(42)
latents = torch.randn((batch_size, unet.in_channels, height//8, width//8))
latents = latents.to('cuda').half()
latents.shape

We initialize the scheduler with `num_inference_steps`. This will prepare the internal state to be used during denoising.


In [ ]:
scheduler.set_timesteps(num_inference_steps)

We scale the latents with the initial standard deviation required by the scheduler.


In [ ]:
latents = latents * scheduler.init_noise_sigma

We are now ready to write the denoising loop. The timesteps go from `999` to `0` following a given schedule.


In [ ]:
scheduler.timesteps

In [ ]:
scheduler.sigmas

In [ ]:
plt.plot(scheduler.timesteps, scheduler.sigmas[:-1])

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for i, t in enumerate(tqdm(scheduler.timesteps)):
    input = torch.cat([latents]*2)
    input = scheduler.scale_model_input(input, t)

    #predict the noise residual
    with torch.no_grad():
        pred = unet(input, t, encoder_hidden_states=text_embeddings).sample

    #perform guidance
    pred_text, pred_uncond = pred.chunk(2)
    pred = pred_uncond + guidance_scale * (pred_text - pred_uncond)

    #compute the previous 'noisy' sample
    latents = scheduler.step(pred, t, latents).prev_sample

Now, our latents contain the denoised representation of the image. We use the `vae` decoder to convert it back to the pixel-space.


In [ ]:
with torch.no_grad(): image = vae.decode(1 / 0.18215 * latents).sample

And finally, lets convert it back to `PIL` to display the image.

The arithmetic acrobats are done to convert the decoded image in a form that can be used correctly by the `PIL`.


In [ ]:
image = (image / 2 + 0.5).clamp(0, 1)
image = image[0].detach().cpu().permute(1, 2, 0).numpy()
image = (image * 255).round().astype("uint8")
Image.fromarray(image)

Voila! Here's our Batman with his coffee.

This was the text2image pipeline. Now, let's build the img2img pipeline. Instead of starting with pure noise, we'll start with a certain image and add noise to it. We skip the initial steps of the denoising process and pretend that the given image is what the algorithm came up with.


In [ ]:
!curl --output philosophical_dog.jpg 'https://puppytoob.com/wp-content/uploads/2013/04/Dogs_In_Wind_8.jpg'

In [ ]:
input_image = Image.open('philosophical_dog.jpg').resize((512, 512))
input_image

Let's create a function to encode our image into the latent space using `vae`.


In [ ]:
from torchvision import transforms as tfms

In [ ]:
def pil_to_latent(input_im):
    # Single image -> single latent in a batch (so size 1, 4, 64, 64)
    with torch.no_grad():
        latent = vae.encode(tfms.ToTensor()(input_im).unsqueeze(0).to('cuda').half()*2 - 1)
    return 0.18215 * latent.latent_dist.sample()

... and a function to get the image back from latent representation


In [ ]:
def latents_to_pil(latents):
    # batch of latents -> list of images
    latents = (1 / 0.18215) * latents
    with torch.no_grad():
        image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images

In [ ]:
encoded = pil_to_latent(input_image)
encoded.shape

Now, let's repeat the process we did above, except that we will start with the input image and not with pure noise.


In [ ]:
#set parameters
prompt = ['A painting in the style of Vincent Van Gogh']
height = 512
width = 512
num_inference_steps = 70
guidance_scale = 7.5
batch_size = 1

# Prep text embeddings
text_input = tokenizer(prompt, padding='max_length', max_length=tokenizer.model_max_length, truncation=True, return_tensors='pt')
text_embeddings = text_encoder(text_input.input_ids.to('cuda'))[0].half()
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer([""]*batch_size, padding='max_length', max_length=max_length, truncation=True, return_tensors='pt')
uncond_embeddings = text_encoder(uncond_input.input_ids.to('cuda'))[0].half()
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

#Prep scheduler (set num of inference steps)
scheduler.set_timesteps(num_inference_steps)

#Prep latents
start_step = 30
start_sigma = scheduler.sigmas[start_step]
noise = torch.randn_like(encoded)
latents = scheduler.add_noise(encoded, noise, timesteps=torch.tensor([scheduler.timesteps[start_step]]))
latents = latents.to('cuda').half()

#Denoising loop
for i, t in enumerate(tqdm(scheduler.timesteps)):
    if i > start_step:
        input = torch.concat([latents]*2)
        input = scheduler.scale_model_input(input, t)

        #predict noise residual
        with torch.no_grad():
            pred = unet(input, t, encoder_hidden_states=text_embeddings).sample

        #perform guidance
        pred_uncond, pred_text = pred.chunk(2)
        pred = pred_uncond + guidance_scale * (pred_text - pred_uncond)

        #compute prev noisy sample
        latents = scheduler.step(pred, t, latents).prev_sample

latents_to_pil(latents)[0]

And here's our dog, ruminating about the meaninglessness of life.
